# **Example of *Paper***

In this notebook we provide the code that can be used to obtain the results from the *paper* in Section *6* to get the first basis elements of the almost-commuting basis for generic operators $L_3$ and $L_5$ of order 3 and 5, respectively.

This implementation is done for these particular cases, and they show intermidiate steps. The complete process can be seen in the method `almost_commuting_wilson` in `dalgebra.commutators.almost_commuting`.

In [1]:
import sys
sys.path.insert(0, "..")
from dalgebra import *
%display latex

In [2]:
import re
from functools import reduce
def poly_to_latex(poly):
    r'''
        Transform a polynomial to a proper LaTeX representation.
        
        We assume the polynomial has variables `u_i`, `c_{i,j}` and `y_i` where the variables `u_i` and `y_i` are differential variables.
        The differential variables are given with `y_i^{(j)}` --  "yi_j".
        The constant variables are given with `c_{i,j}` -- "c_i_j".
        
        This method also guarantees the removal of "*" for multiplications and the "+ (-...)".
    '''
    if not isinstance(poly, str):
        poly = str(poly)
        
    patterns = list()
    
    patterns.append((r"\*", "")) # We remove the multiplication symbol
    patterns.append((r"\+ \(-([^\)])+\)", lambda M: f"- {M.groups()[0]}")) # We remove the + (-number)
    patterns.append((r"\(-([^\)])+\)", lambda M: f"-{M.groups()[0]}")) # We remove the + (-number)
    patterns.append((r"(\d+)/(\d+)", lambda M: f"\\frac\u007b{M.groups()[0]}\u007d\u007b{M.groups()[1]}\u007d"))
    patterns.append((r"c_(\d+)_(\d+)", lambda M: f"c_\u007b{M.groups()[0]},{M.groups()[1]}\u007d")) # Rewritting constant variables
    ## Rewriting differential variables
    patterns.append((r"([uy])(\d+)_0([^\d]?)", lambda M: f"{M.groups()[0]}_\u007b{M.groups()[1]}\u007d{M.groups()[2]}"))
    patterns.append((r"([uy])(\d+)_1([^\d]?)", lambda M: f"{M.groups()[0]}_\u007b{M.groups()[1]}\u007d'{M.groups()[2]}"))
    patterns.append((r"([uy])(\d+)_2([^\d]?)", lambda M: f"{M.groups()[0]}_\u007b{M.groups()[1]}\u007d''{M.groups()[2]}"))
    patterns.append((r"([uy])(\d+)_3([^\d]?)", lambda M: f"{M.groups()[0]}_\u007b{M.groups()[1]}\u007d'''{M.groups()[2]}"))
    patterns.append((r"([uy])(\d+)_(\d+)", lambda M: f"{M.groups()[0]}_\u007b{M.groups()[1]}\u007d^\u007b({M.groups()[2]})\u007d"))
    patterns.append((r"([uy])(\d+)", lambda M: f"{M.groups()[0]}_\u007b{M.groups()[1]}\u007d"))
    patterns.append((r"z_(\d+)", lambda M: f"\\partial^\u007b{M.groups()[0]}\u007d"))
    
    return reduce(lambda p, q : re.sub(q[0], q[1], p), patterns, poly)

## Case of order 3

In [3]:
## Setting up the framework to work
R.<u2,u3,y2,y3,y4,y5,z> = DifferentialPolynomialRing(QQ)
u = (1, 0, u2, u3)
y = (1, 0, y2, y3, y4, y5)
L = sum(z[i]*u[3-i] for i in range(4))
## Showing L
LatexExpr(poly_to_latex(L))

u_{2}\partial^{1} + u_{3}\partial^{0} + \partial^{3}

In [4]:
## Creating the \tilde{P}_i polynomials
PT = [1, z[1]]
while(len(PT) <= 5): # creating up to P_6(U;Y)
    PT += [PT[-1](z=z[1]) + y[len(PT)]*z[0]]

## Showing the PT
for (i,p) in enumerate(PT):
    show(LatexExpr(r"\tilde{P}_{" + str(i) + r"}(U,Y) \longrightarrow " + poly_to_latex(p)))

\tilde{P}_{0}(U,Y) \longrightarrow 1

\tilde{P}_{1}(U,Y) \longrightarrow \partial^{1}

\tilde{P}_{2}(U,Y) \longrightarrow y_{2}\partial^{0} + \partial^{2}

\tilde{P}_{3}(U,Y) \longrightarrow y_{2}\partial^{1} + y_{3}\partial^{0} + \partial^{3}

\tilde{P}_{4}(U,Y) \longrightarrow y_{2}\partial^{2} + y_{3}\partial^{1} + y_{4}\partial^{0} + \partial^{4}

\tilde{P}_{5}(U,Y) \longrightarrow y_{2}\partial^{3} + y_{3}\partial^{2} + y_{4}\partial^{1} + y_{5}\partial^{0} + \partial^{5}

#### **1. Computing the differential systems**

In [5]:
lie_brackets = [L.lie_bracket(P, "z") for P in PT]

In [6]:
diff_systems = [tuple([bracket.coefficient(z[i]) for i in range(L.order(z)-1, bracket.order(z)+1)]) for bracket in lie_brackets]

##### The system for $m = 1$

In [7]:
bracket = lie_brackets[1]
LatexExpr(r"[L_3, \tilde{P}_1(U)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^1+\left(-u_{3}'\right)

In [8]:
print(_)

[L_3, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^1+\left(-u_{3}'\right)


In [9]:
## There are no equations for the almost commuting of z[1]
##  --> trivial: it is the unique monic operator of order 1 in normal form
diff_systems[1]

()

##### The system for $m=2$

In [10]:
bracket = lie_brackets[2]
LatexExpr(r"[L_3, \tilde{P}_2(U, y_2)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 3y_{2}'\right)\partial^2+\left(-u_{2}'' - 2u_{3}' + 3y_{2}''\right)\partial^1+\left(u_{2}y_{2}' - u_{3}'' + y_{2}'''\right)

In [11]:
print(_)

[L_3, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 3y_{2}'\right)\partial^2+\left(-u_{2}'' - 2u_{3}' + 3y_{2}''\right)\partial^1+\left(u_{2}y_{2}' - u_{3}'' + y_{2}'''\right)


In [12]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[2]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [13]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 3$

In [14]:
bracket = lie_brackets[3]
LatexExpr(r"[L_3, \tilde{P}_3(U, y_2, y_3)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 3y_{2}'\right)\partial^3+\left(-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^2+\left(-u_{2}'y_{2} + u_{2}y_{2}' - u_{2}''' - 3u_{3}'' + y_{2}''' + 3y_{3}''\right)\partial^1+\left(u_{2}y_{3}' - u_{3}'y_{2} - u_{3}''' + y_{3}'''\right)

In [15]:
print(_)

[L_3, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 3y_{2}'\right)\partial^3+\left(-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^2+\left(-u_{2}'y_{2} + u_{2}y_{2}' - u_{2}''' - 3u_{3}'' + y_{2}''' + 3y_{3}''\right)\partial^1+\left(u_{2}y_{3}' - u_{3}'y_{2} - u_{3}''' + y_{3}'''\right)


In [16]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[3]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-3u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [17]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-3u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 4$

In [18]:
bracket = lie_brackets[4]
LatexExpr(r"[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 3y_{2}'\right)\partial^4+\left(-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^3+\left(-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^2+\left(-u_{2}''y_{2} - u_{2}'y_{3} + u_{2}y_{3}' - 2u_{3}'y_{2} - u_{2}^{(4)} - 4u_{3}''' + y_{3}''' + 3y_{4}''\right)\partial^1+\left(u_{2}y_{4}' - u_{3}''y_{2} - u_{3}'y_{3} - u_{3}^{(4)} + y_{4}'''\right)

In [19]:
print(_)

[L_3, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 3y_{2}'\right)\partial^4+\left(-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^3+\left(-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^2+\left(-u_{2}''y_{2} - u_{2}'y_{3} + u_{2}y_{3}' - 2u_{3}'y_{2} - u_{2}^{(4)} - 4u_{3}''' + y_{3}''' + 3y_{4}''\right)\partial^1+\left(u_{2}y_{4}' - u_{3}''y_{2} - u_{3}'y_{3} - u_{3}^{(4)} + y_{4}'''\right)


In [20]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[4]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-4u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [21]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}'y_{2} + u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-4u_{2}' + 3y_{2}'&= 0
\end{array}\right.


##### The system for $m = 5$

In [22]:
bracket = lie_brackets[5]
LatexExpr(r"[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 3y_{2}'\right)\partial^5+\left(-0u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^4+\left(-3u_{2}'y_{2} + u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^3+\left(-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 0u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'\right)\partial^2+\left(-u_{2}'''y_{2} - u_{2}''y_{3} - u_{2}'y_{4} + u_{2}y_{4}' - 3u_{3}''y_{2} - 2u_{3}'y_{3} - u_{2}^{(5)} - 5u_{3}^{(4)} + y_{4}''' + 3y_{5}''\right)\partial^1+\left(u_{2}y_{5}' - u_{3}'''y_{2} - u_{3}''y_{3} - u_{3}'y_{4} - u_{3}^{(5)} + y_{5}'''\right)

In [23]:
print(_)

[L_3, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 3y_{2}'\right)\partial^5+\left(-0u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'\right)\partial^4+\left(-3u_{2}'y_{2} + u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'\right)\partial^3+\left(-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 0u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'\right)\partial^2+\left(-u_{2}'''y_{2} - u_{2}''y_{3} - u_{2}'y_{4} + u_{2}y_{4}' - 3u_{3}''y_{2} - 2u_{3}'y_{3} - u_{2}^{(5)} - 5u_{3}^{(4)} + y_{4}''' + 3y_{5}''\right)\partial^1+\left(u_{2}y_{5}' - u_{3}'''y_{2} - u_{3}''y_{3} - u_{3}'y_{4} - u_{3}^{(5)} + y_{5}'''\right)


In [24]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[5]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 0u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'&= 0\\
	-3u_{2}'y_{2} + u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-0u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-5u_{2}' + 3y_{2}'&= 0
\end{array}\right.

In [25]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}''y_{2} - 2u_{2}'y_{3} + u_{2}y_{3}' - 3u_{3}'y_{2} - 5u_{2}^{(4)} - 0u_{3}''' + y_{3}''' + 3y_{4}'' + 3y_{5}'&= 0\\
	-3u_{2}'y_{2} + u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' + y_{2}''' + 3y_{3}'' + 3y_{4}'&= 0\\
	-0u_{2}'' - 5u_{3}' + 3y_{2}'' + 3y_{3}'&= 0\\
	-5u_{2}' + 3y_{2}'&= 0
\end{array}\right.


#### **2. Computing the basis of $W(L_5)$**

In [26]:
W = R.weight_func([2,3,0,0,0,0,0], [1])
T = [W.homogeneous_monomials(i) for i in range(6)]

In [27]:
for i in range(6):
    show(LatexExpr(r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}"))

\mathcal{T}_{0} = \{1\}

\mathcal{T}_{1} = \{\}

\mathcal{T}_{2} = \{u_{2}\}

\mathcal{T}_{3} = \{u_{2}',u_{3}\}

\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2\}

\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}'u_{2},u_{2}u_{3}^{(0)}\}

In [28]:
print(LatexExpr(
    r"\begin{itemize}" + "\n\t" + r"\item $" + ("\n\t" + r"\item $").join(
        r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}$"
    for i in range(6)) + "\n" + r"\end{itemize}"))

\begin{itemize}
	\item $\mathcal{T}_{0} = \{1\}$
	\item $\mathcal{T}_{1} = \{\}$
	\item $\mathcal{T}_{2} = \{u_{2}\}$
	\item $\mathcal{T}_{3} = \{u_{2}',u_{3}\}$
	\item $\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2\}$
	\item $\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}'u_{2},u_{2}u_{3}^{(0)}\}$
\end{itemize}


In [29]:
## We create now the constants
RwC = DifferentialRing(PolynomialRing(QQ, sum(([f"c_{i}_{j}" for j in range(len(T[i]))] for i in range(2,6)), [])))
C = [[],[]] + [[RwC(f"c_{i}_{j}") for j in range(len(T[i]))] for i in range(2,6)]

In [30]:
y_subs = {f"y{i}" : sum(c*t for (c,t) in zip(C[i], T[i])) for i in range(2,6)}
print(LatexExpr(
    r"\begin{array}{rcl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(k) + r" & \rightarrow & " + poly_to_latex(v) for (k,v) in y_subs.items()) + "\n" + r"\end{array}"))

\begin{array}{rcl}
	y_{2} & \rightarrow & c_{2,0}u_{2}\\
	y_{3} & \rightarrow & c_{3,0}u_{2}' + c_{3,1}u_{3}\\
	y_{4} & \rightarrow & c_{4,2}u_{2}^2 + c_{4,0}u_{2}'' + c_{4,1}u_{3}'\\
	y_{5} & \rightarrow & c_{5,2}u_{2}'u_{2} + c_{5,3}u_{2}u_{3}^{(0)} + c_{5,0}u_{2}''' + c_{5,1}u_{3}''
\end{array}


In [31]:
## We compute the linear system for each $m$
lin_systems = [sum(([RwC.wrapped(c) for c in el(**y_subs).coefficients()] for el in diff_systems[m]), []) for m in range(6)]

In [32]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[2]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{2,0} & = -2
\end{array}\right.

In [33]:
print(_)

\left\{\begin{array}{rl}
	3c_{2,0} & = -2
\end{array}\right.


In [34]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[3]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{2,0} + 3c_{3,0} & = -3\\
	3c_{3,1} & = -3\\
	3c_{2,0} & = -3
\end{array}\right.

In [35]:
print(_)

\left\{\begin{array}{rl}
	3c_{2,0} + 3c_{3,0} & = -3\\
	3c_{3,1} & = -3\\
	3c_{2,0} & = -3
\end{array}\right.


In [36]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[4]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	-c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -4\\
	3c_{3,1} + 3c_{4,1} & = -6\\
	3c_{2,0} + 3c_{3,0} & = -6\\
	3c_{3,1} & = -4\\
	3c_{2,0} & = -4
\end{array}\right.

In [37]:
print(_)

\left\{\begin{array}{rl}
	-c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -4\\
	3c_{3,1} + 3c_{4,1} & = -6\\
	3c_{2,0} + 3c_{3,0} & = -6\\
	3c_{3,1} & = -4\\
	3c_{2,0} & = -4
\end{array}\right.


In [38]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[5]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3c_{2,0} + c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,1} + 3c_{5,3} & = 0\\
	-3c_{2,0} + c_{3,1} + 3c_{5,3} & = 0\\
	c_{3,0} + 3c_{4,0} + 3c_{5,0} & = -5\\
	c_{3,1} + 3c_{4,1} + 3c_{5,1} & = -10\\
	-2c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -10\\
	3c_{3,1} + 3c_{4,1} & = -10\\
	3c_{2,0} + 3c_{3,0} & = -10\\
	3c_{3,1} & = -5\\
	3c_{2,0} & = -5
\end{array}\right.

In [39]:
print(_)

\left\{\begin{array}{rl}
	-3c_{2,0} + c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,0} + 6c_{4,2} + 3c_{5,2} & = 0\\
	-2c_{3,1} + 3c_{5,3} & = 0\\
	-3c_{2,0} + c_{3,1} + 3c_{5,3} & = 0\\
	c_{3,0} + 3c_{4,0} + 3c_{5,0} & = -5\\
	c_{3,1} + 3c_{4,1} + 3c_{5,1} & = -10\\
	-2c_{2,0} + 6c_{4,2} & = 0\\
	c_{2,0} + 3c_{3,0} + 3c_{4,0} & = -10\\
	3c_{3,1} + 3c_{4,1} & = -10\\
	3c_{2,0} + 3c_{3,0} & = -10\\
	3c_{3,1} & = -5\\
	3c_{2,0} & = -5
\end{array}\right.


In [40]:
solutions = [ideal(system).groebner_basis() if len(system) > 0 else [] for system in lin_systems]

In [41]:
for i in range(len(solutions)):
    show(i, LatexExpr(r"\qquad\longrightarrow\qquad"), [LatexExpr(poly_to_latex(sol)) for sol in solutions[i]])

0 \qquad\longrightarrow\qquad []

1 \qquad\longrightarrow\qquad []

2 \qquad\longrightarrow\qquad [c_{2,0} - \frac{2}{3}]

3 \qquad\longrightarrow\qquad [c_{2,0} - 1, c_{3,0}, c_{3,1} - 1]

4 \qquad\longrightarrow\qquad [c_{2,0} - \frac{4}{3},
 c_{3,0} - \frac{2}{3},
 c_{3,1} - \frac{4}{3},
 c_{4,0} - \frac{2}{9},
 c_{4,1} - \frac{2}{3},
 c_{4,2} - \frac{2}{9}]

5 \qquad\longrightarrow\qquad [c_{2,0} - \frac{5}{3},
 c_{3,0} - \frac{5}{3},
 c_{3,1} - \frac{5}{3},
 c_{4,0} - \frac{10}{9},
 c_{4,1} - \frac{5}{3},
 c_{4,2} - \frac{5}{9},
 c_{5,0},
 c_{5,1} - \frac{10}{9},
 c_{5,2},
 c_{5,3} - \frac{10}{9}]

#### **3. Computing the hierarchy equations**

In [42]:
y_subs_true = [dict(),dict()] + [{f"y{i}" : sum(QQ(RwC.wrapped(c).reduce(solutions[m]))*t for (c,t) in zip(C[i], T[i])) for i in range(2,m+1)} for m in range(2,6)]

In [43]:
H = [[lie_brackets[m].coefficient(z[i])(**y_subs_true[m]) for i in range(L.order(z)-1)] for m in range(6)]

##### m = 0

In [44]:
m = 0
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3) = & 0\\
	H_{0,1}(u_2,u_3) = & 0
\end{array}\right.

In [45]:
print(_)

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3) = & 0\\
	H_{0,1}(u_2,u_3) = & 0
\end{array}\right.


##### m = 1

In [46]:
m=1
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3) = & -u_{3}'\\
	H_{1,1}(u_2,u_3) = & -u_{2}'
\end{array}\right.

In [47]:
print(_)

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3) = & -u_{3}'\\
	H_{1,1}(u_2,u_3) = & -u_{2}'
\end{array}\right.


##### m = 2

In [48]:
m=2
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3) = & \frac{2}{3}u_{2}'u_{2} + \frac{2}{3}u_{2}''' - u_{3}''\\
	H_{2,1}(u_2,u_3) = & u_{2}'' - 2u_{3}'
\end{array}\right.

In [49]:
print(_)

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3) = & \frac{2}{3}u_{2}'u_{2} + \frac{2}{3}u_{2}''' - u_{3}''\\
	H_{2,1}(u_2,u_3) = & u_{2}'' - 2u_{3}'
\end{array}\right.


##### m = 3

In [50]:
m=3
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3) = & 0\\
	H_{3,1}(u_2,u_3) = & 0
\end{array}\right.

In [51]:
print(_)

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3) = & 0\\
	H_{3,1}(u_2,u_3) = & 0
\end{array}\right.


##### m = 4

In [52]:
m=4
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3) = & \frac{4}{9}u_{2}'u_{2}^2 + \frac{2}{3}u_{2}'''u_{2} + \frac{4}{3}u_{2}''u_{2}' - 3u_{2}'u_{3}^{(1)} - 3u_{2}u_{3}'' - 3u_{3}'u_{3} + \frac{2}{9}u_{2}^{(5)} - 3u_{3}^{(4)}\\
	H_{4,1}(u_2,u_3) = & \frac{2}{3}u_{2}''u_{2} + \frac{2}{3}u_{2}'^2 - 3u_{2}'u_{3} - 3u_{2}u_{3}' + \frac{1}{3}u_{2}^{(4)} - 3u_{3}'''
\end{array}\right.

In [53]:
print(_)

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3) = & \frac{4}{9}u_{2}'u_{2}^2 + \frac{2}{3}u_{2}'''u_{2} + \frac{4}{3}u_{2}''u_{2}' - 3u_{2}'u_{3}^{(1)} - 3u_{2}u_{3}'' - 3u_{3}'u_{3} + \frac{2}{9}u_{2}^{(5)} - 3u_{3}^{(4)}\\
	H_{4,1}(u_2,u_3) = & \frac{2}{3}u_{2}''u_{2} + \frac{2}{3}u_{2}'^2 - 3u_{2}'u_{3} - 3u_{2}u_{3}' + \frac{1}{3}u_{2}^{(4)} - 3u_{3}'''
\end{array}\right.


##### m = 5

In [54]:
m=5
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3) = & \frac{10}{9}u_{2}'u_{2}u_{3}^{(0)} + \frac{5}{9}u_{2}^2u_{3}' + \frac{10}{9}u_{2}'''u_{3} + \frac{20}{9}u_{2}''u_{3}' + \frac{5}{3}u_{2}'u_{3}'' + \frac{5}{9}u_{2}u_{3}''' - 3u_{3}''u_{3} - 3u_{3}'^2 + \frac{1}{9}u_{3}^{(5)}\\
	H_{5,1}(u_2,u_3) = & \frac{5}{9}u_{2}'u_{2}^2 + \frac{5}{9}u_{2}'''u_{2} + \frac{5}{9}u_{2}''u_{2}' + \frac{5}{3}u_{2}''u_{3} + \frac{5}{3}u_{2}'u_{3}^{(1)} - 3u_{3}'u_{3} + \frac{1}{9}u_{2}^{(5)}
\end{array}\right.

In [55]:
print(_)

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3) = & \frac{10}{9}u_{2}'u_{2}u_{3}^{(0)} + \frac{5}{9}u_{2}^2u_{3}' + \frac{10}{9}u_{2}'''u_{3} + \frac{20}{9}u_{2}''u_{3}' + \frac{5}{3}u_{2}'u_{3}'' + \frac{5}{9}u_{2}u_{3}''' - 3u_{3}''u_{3} - 3u_{3}'^2 + \frac{1}{9}u_{3}^{(5)}\\
	H_{5,1}(u_2,u_3) = & \frac{5}{9}u_{2}'u_{2}^2 + \frac{5}{9}u_{2}'''u_{2} + \frac{5}{9}u_{2}''u_{2}' + \frac{5}{3}u_{2}''u_{3} + \frac{5}{3}u_{2}'u_{3}^{(1)} - 3u_{3}'u_{3} + \frac{1}{9}u_{2}^{(5)}
\end{array}\right.


## Case of order 5

In [56]:
## Setting up the framework to work
R.<u2,u3,u4,u5,y2,y3,y4,y5,y6,z> = DifferentialPolynomialRing(QQ)
u = (1, 0, u2, u3, u4, u5)
y = (1, 0, y2, y3, y4, y5, y6)
L = sum(z[i]*u[5-i] for i in range(6))
## Showing L
LatexExpr(poly_to_latex(L))

u_{2}\partial^{3} + u_{3}\partial^{2} + u_{4}\partial^{1} + u_{5}\partial^{0} + \partial^{5}

In [57]:
## Creating the \tilde{P}_i polynomials
PT = [1, z[1]]
while(len(PT) <= 6): # creating up to P_6(U;Y)
    PT += [PT[-1](z=z[1]) + y[len(PT)]*z[0]]

## Showing the PT
for (i,p) in enumerate(PT):
    show(LatexExpr(r"\tilde{P}_{" + str(i) + r"}(U,Y) \longrightarrow " + poly_to_latex(p)))

\tilde{P}_{0}(U,Y) \longrightarrow 1

\tilde{P}_{1}(U,Y) \longrightarrow \partial^{1}

\tilde{P}_{2}(U,Y) \longrightarrow y_{2}\partial^{0} + \partial^{2}

\tilde{P}_{3}(U,Y) \longrightarrow y_{2}\partial^{1} + y_{3}\partial^{0} + \partial^{3}

\tilde{P}_{4}(U,Y) \longrightarrow y_{2}\partial^{2} + y_{3}\partial^{1} + y_{4}\partial^{0} + \partial^{4}

\tilde{P}_{5}(U,Y) \longrightarrow y_{2}\partial^{3} + y_{3}\partial^{2} + y_{4}\partial^{1} + y_{5}\partial^{0} + \partial^{5}

\tilde{P}_{6}(U,Y) \longrightarrow y_{2}\partial^{4} + y_{3}\partial^{3} + y_{4}\partial^{2} + y_{5}\partial^{1} + y_{6}\partial^{0} + \partial^{6}

#### **1. Computing the differential systems**

In [58]:
lie_brackets = [L.lie_bracket(P, "z") for P in PT]

In [59]:
diff_systems = [tuple([bracket.coefficient(z[i]) for i in range(L.order(z)-1, bracket.order(z)+1)]) for bracket in lie_brackets]

##### The system for $m = 1$

In [60]:
bracket = lie_brackets[1]
LatexExpr(r"[L_5, \tilde{P}_1(U)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_5, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^3+\left(-u_{3}'\right)\partial^2+\left(-u_{4}'\right)\partial^1+\left(-u_{5}'\right)

In [61]:
print(_)

[L_5, \tilde{P}_1(U)] = \left(-u_{2}'\right)\partial^3+\left(-u_{3}'\right)\partial^2+\left(-u_{4}'\right)\partial^1+\left(-u_{5}'\right)


In [62]:
## There are no equations for the almost commuting of z[1]
##  --> trivial: it is the unique monic operator of order 1 in normal form
diff_systems[1]

()

##### The system for $m=2$

In [63]:
bracket = lie_brackets[2]
LatexExpr(r"[L_5, \tilde{P}_2(U, y_2)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_5, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 5y_{2}'\right)\partial^4+\left(-u_{2}'' - 2u_{3}' + 10y_{2}''\right)\partial^3+\left(3u_{2}y_{2}' - u_{3}'' - 2u_{4}' + 10y_{2}'''\right)\partial^2+\left(3u_{2}y_{2}'' + 2u_{3}y_{2}' - u_{4}'' - 2u_{5}' + 5y_{2}^{(4)}\right)\partial^1+\left(u_{2}y_{2}''' + u_{3}y_{2}'' + u_{4}y_{2}' - u_{5}'' + y_{2}^{(5)}\right)

In [64]:
print(_)

[L_5, \tilde{P}_2(U, y_2)] = \left(-2u_{2}' + 5y_{2}'\right)\partial^4+\left(-u_{2}'' - 2u_{3}' + 10y_{2}''\right)\partial^3+\left(3u_{2}y_{2}' - u_{3}'' - 2u_{4}' + 10y_{2}'''\right)\partial^2+\left(3u_{2}y_{2}'' + 2u_{3}y_{2}' - u_{4}'' - 2u_{5}' + 5y_{2}^{(4)}\right)\partial^1+\left(u_{2}y_{2}''' + u_{3}y_{2}'' + u_{4}y_{2}' - u_{5}'' + y_{2}^{(5)}\right)


In [65]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[2]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [66]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 3$

In [67]:
bracket = lie_brackets[3]
LatexExpr(r"[L_5, \tilde{P}_3(U, y_2, y_3)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_5, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 5y_{2}'\right)\partial^5+\left(-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^4+\left(-u_{2}'y_{2} + 3u_{2}y_{2}' - u_{2}''' - 3u_{3}'' - 3u_{4}' + 10y_{2}''' + 10y_{3}''\right)\partial^3+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - u_{3}'y_{2} + 2u_{3}y_{2}' - u_{3}''' - 3u_{4}'' - 3u_{5}' + 5y_{2}^{(4)} + 10y_{3}'''\right)\partial^2+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + u_{3}y_{2}'' + 2u_{3}y_{3}' - u_{4}'y_{2} + u_{4}y_{2}' - u_{4}''' - 3u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)}\right)\partial^1+\left(u_{2}y_{3}''' + u_{3}y_{3}'' + u_{4}y_{3}' - u_{5}'y_{2} - u_{5}''' + y_{3}^{(5)}\right)

In [68]:
print(_)

[L_5, \tilde{P}_3(U, y_2, y_3)] = \left(-3u_{2}' + 5y_{2}'\right)\partial^5+\left(-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^4+\left(-u_{2}'y_{2} + 3u_{2}y_{2}' - u_{2}''' - 3u_{3}'' - 3u_{4}' + 10y_{2}''' + 10y_{3}''\right)\partial^3+\left(3u_{2}y_{2}'' + 3u_{2}y_{3}' - u_{3}'y_{2} + 2u_{3}y_{2}' - u_{3}''' - 3u_{4}'' - 3u_{5}' + 5y_{2}^{(4)} + 10y_{3}'''\right)\partial^2+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + u_{3}y_{2}'' + 2u_{3}y_{3}' - u_{4}'y_{2} + u_{4}y_{2}' - u_{4}''' - 3u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)}\right)\partial^1+\left(u_{2}y_{3}''' + u_{3}y_{3}'' + u_{4}y_{3}' - u_{5}'y_{2} - u_{5}''' + y_{3}^{(5)}\right)


In [69]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[3]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-3u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [70]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}'' - 3u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-3u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 4$

In [71]:
bracket = lie_brackets[4]
LatexExpr(r"[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 5y_{2}'\right)\partial^6+\left(-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^5+\left(-2u_{2}'y_{2} + 3u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^4+\left(-u_{2}''y_{2} - u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 2u_{3}'y_{2} + 2u_{3}y_{2}' - u_{2}^{(4)} - 4u_{3}''' - 6u_{4}'' - 4u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}''\right)\partial^3+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - u_{3}''y_{2} - u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 2u_{4}'y_{2} + u_{4}y_{2}' - u_{3}^{(4)} - 4u_{4}''' - 6u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}'''\right)\partial^2+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + u_{3}y_{3}'' + 2u_{3}y_{4}' - u_{4}''y_{2} - u_{4}'y_{3} + u_{4}y_{3}' - 2u_{5}'y_{2} - u_{4}^{(4)} - 4u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)}\right)\partial^1+\left(u_{2}y_{4}''' + u_{3}y_{4}'' + u_{4}y_{4}' - u_{5}''y_{2} - u_{5}'y_{3} - u_{5}^{(4)} + y_{4}^{(5)}\right)

In [72]:
print(_)

[L_5, \tilde{P}_4(U, y_2,y_3,y_4)] = \left(-4u_{2}' + 5y_{2}'\right)\partial^6+\left(-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^5+\left(-2u_{2}'y_{2} + 3u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^4+\left(-u_{2}''y_{2} - u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 2u_{3}'y_{2} + 2u_{3}y_{2}' - u_{2}^{(4)} - 4u_{3}''' - 6u_{4}'' - 4u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}''\right)\partial^3+\left(u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - u_{3}''y_{2} - u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 2u_{4}'y_{2} + u_{4}y_{2}' - u_{3}^{(4)} - 4u_{4}''' - 6u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}'''\right)\partial^2+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + u_{3}y_{3}'' + 2u_{3}y_{4}' - u_{4}''y_{2} - u_{4}'y_{3} + u_{4}y_{3}' - 2u_{5}'y_{2} - u_{4}^{(4)} - 4u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)}\right)\partial^1+\left(u_{2}y_{4}''' + u_{3}y_{4}'' + u_{4}y_{4}' - u_{5}''y_{2} - u_{5}'y_{3} - u_{5}^{(4)} + y_{4}^{(5)}\

In [73]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[4]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-2u_{2}'y_{2} + 3u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-4u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [74]:
print(_)

\left\{\begin{array}{rl}
	-2u_{2}'y_{2} + 3u_{2}y_{2}' - 4u_{2}''' - 6u_{3}'' - 4u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-6u_{2}'' - 4u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-4u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 5$

In [75]:
bracket = lie_brackets[5]
LatexExpr(r"[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 5y_{2}'\right)\partial^7+\left(-0u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^6+\left(-3u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^5+\left(-3u_{2}''y_{2} - 2u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 0u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^4+\left(-u_{2}'''y_{2} - u_{2}''y_{3} - u_{2}'y_{4} + u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 3u_{3}''y_{2} - 2u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 3u_{4}'y_{2} + u_{4}y_{2}' - u_{2}^{(5)} - 5u_{3}^{(4)} - 0u_{4}''' - 0u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}''\right)\partial^3+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' - u_{3}'''y_{2} - u_{3}''y_{3} - u_{3}'y_{4} + u_{3}y_{3}'' + 2u_{3}y_{4}' - 3u_{4}''y_{2} - 2u_{4}'y_{3} + u_{4}y_{3}' - 3u_{5}'y_{2} - u_{3}^{(5)} - 5u_{4}^{(4)} - 0u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)} + 10y_{5}'''\right)\partial^2+\left(u_{2}y_{4}''' + 3u_{2}y_{5}'' + u_{3}y_{4}'' + 2u_{3}y_{5}' - u_{4}'''y_{2} - u_{4}''y_{3} - u_{4}'y_{4} + u_{4}y_{4}' - 3u_{5}''y_{2} - 2u_{5}'y_{3} - u_{4}^{(5)} - 5u_{5}^{(4)} + y_{4}^{(5)} + 5y_{5}^{(4)}\right)\partial^1+\left(u_{2}y_{5}''' + u_{3}y_{5}'' + u_{4}y_{5}' - u_{5}'''y_{2} - u_{5}''y_{3} - u_{5}'y_{4} - u_{5}^{(5)} + y_{5}^{(5)}\right)

In [76]:
print(_)

[L_5, \tilde{P}_5(U, y_2,y_3,y_4,y_5)] = \left(-5u_{2}' + 5y_{2}'\right)\partial^7+\left(-0u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^6+\left(-3u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^5+\left(-3u_{2}''y_{2} - 2u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 0u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^4+\left(-u_{2}'''y_{2} - u_{2}''y_{3} - u_{2}'y_{4} + u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 3u_{3}''y_{2} - 2u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 3u_{4}'y_{2} + u_{4}y_{2}' - u_{2}^{(5)} - 5u_{3}^{(4)} - 0u_{4}''' - 0u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}''\right)\partial^3+\left(u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' - u_{3}'''y_{2} - u_{3}''y_{3} - u_{3}'y_{4} + u_{3}y_{3}'' + 2u_{3}y_{4}' - 3u_{4}''y_{2} - 2u_{4}'y_{3} + u_{4}y_{3}' - 3u_{5}'y_{2} - u_{3}^{(5)} - 5u_{4}^{

In [77]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[5]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-3u_{2}''y_{2} - 2u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 0u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	-3u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-0u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-5u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [78]:
print(_)

\left\{\begin{array}{rl}
	-3u_{2}''y_{2} - 2u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 3u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 0u_{4}'' - 5u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	-3u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 0u_{3}'' - 5u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-0u_{2}'' - 5u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-5u_{2}' + 5y_{2}'&= 0
\end{array}\right.


##### The system for $m = 6$

In [79]:
bracket = lie_brackets[6]
LatexExpr(r"[L_5, \tilde{P}_6(U, Y)] = " + "+".join(r"\left(" + poly_to_latex(bracket.coefficient(z[i])) + r"\right)" + (r"\partial^" + str(i) if i > 0 else "") for i in range(bracket.order(z), -1, -1) if bracket.coefficient(z[i]) != 0))

[L_5, \tilde{P}_6(U, Y)] = \left(-6u_{2}' + 5y_{2}'\right)\partial^8+\left(-5u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^7+\left(-4u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 5u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^6+\left(-6u_{2}''y_{2} - 3u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 4u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 5u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^5+\left(-4u_{2}'''y_{2} - 3u_{2}''y_{3} - 2u_{2}'y_{4} + u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 6u_{3}''y_{2} - 3u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 4u_{4}'y_{2} + u_{4}y_{2}' - 6u_{2}^{(5)} - 5u_{3}^{(4)} - 0u_{4}''' - 5u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'\right)\partial^4+\left(-u_{2}^{(4)}y_{2} - u_{2}'''y_{3} - u_{2}''y_{4} - u_{2}'y_{5} + u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' - 4u_{3}'''y_{2} - 3u_{3}''y_{3} - 2u_{3}'y_{4} + u_{3}y_{3}'' + 2u_{3}y_{4}' - 6u_{4}''y_{2} - 3u_{4}'y_{3} + u_{4}y_{3}' - 4u_{5}'y_{2} - u_{2}^{(6)} - 6u_{3}^{(5)} - 5u_{4}^{(4)} - 0u_{5}''' + y_{3}^{(5)} + 5y_{4}^{(4)} + 10y_{5}''' + 10y_{6}''\right)\partial^3+\left(u_{2}y_{4}''' + 3u_{2}y_{5}'' + 3u_{2}y_{6}' - u_{3}^{(4)}y_{2} - u_{3}'''y_{3} - u_{3}''y_{4} - u_{3}'y_{5} + u_{3}y_{4}'' + 2u_{3}y_{5}' - 4u_{4}'''y_{2} - 3u_{4}''y_{3} - 2u_{4}'y_{4} + u_{4}y_{4}' - 6u_{5}''y_{2} - 3u_{5}'y_{3} - u_{3}^{(6)} - 6u_{4}^{(5)} - 5u_{5}^{(4)} + y_{4}^{(5)} + 5y_{5}^{(4)} + 10y_{6}'''\right)\partial^2+\left(u_{2}y_{5}''' + 3u_{2}y_{6}'' + u_{3}y_{5}'' + 2u_{3}y_{6}' - u_{4}^{(4)}y_{2} - u_{4}'''y_{3} - u_{4}''y_{4} - u_{4}'y_{5} + u_{4}y_{5}' - 4u_{5}'''y_{2} - 3u_{5}''y_{3} - 2u_{5}'y_{4} - u_{4}^{(6)} - 6u_{5}^{(5)} + y_{5}^{(5)} + 5y_{6}^{(4)}\right)\partial^1+\left(u_{2}y_{6}''' + u_{3}y_{6}'' + u_{4}y_{6}' - u_{5}^{(4)}y_{2} - u_{5}'''y_{3} - u_{5}''y_{4} - u_{5}'y_{5} - u_{5}^{(6)} + y_{6}^{(5)}\right)

In [80]:
print(_)

[L_5, \tilde{P}_6(U, Y)] = \left(-6u_{2}' + 5y_{2}'\right)\partial^8+\left(-5u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'\right)\partial^7+\left(-4u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 5u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'\right)\partial^6+\left(-6u_{2}''y_{2} - 3u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 4u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 5u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'\right)\partial^5+\left(-4u_{2}'''y_{2} - 3u_{2}''y_{3} - 2u_{2}'y_{4} + u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 6u_{3}''y_{2} - 3u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 4u_{4}'y_{2} + u_{4}y_{2}' - 6u_{2}^{(5)} - 5u_{3}^{(4)} - 0u_{4}''' - 5u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'\right)\partial^4+\left(-u_{2}^{(4)}y_{2} - u_{2}'''y_{3} - u_{2}''y_{4} - u_{2}'y_{5} + u_{2}y_{3}''' + 3u_{2}y_{4}'' + 3u_{2}y_{5}' - 4u_{3}'''y_{2} - 3u_{3}''y_{3} - 2u_{3}'y_{4} + u_{3}y_{3}'' + 2u_{3}y_{4}' - 6u_{4}''y_{2} 

In [81]:
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(p) + r"&= 0" for p in diff_systems[6]) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	-4u_{2}'''y_{2} - 3u_{2}''y_{3} - 2u_{2}'y_{4} + u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 6u_{3}''y_{2} - 3u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 4u_{4}'y_{2} + u_{4}y_{2}' - 6u_{2}^{(5)} - 5u_{3}^{(4)} - 0u_{4}''' - 5u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'&= 0\\
	-6u_{2}''y_{2} - 3u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 4u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 5u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	-4u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 5u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-5u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-6u_{2}' + 5y_{2}'&= 0
\end{array}\right.

In [82]:
print(_)

\left\{\begin{array}{rl}
	-4u_{2}'''y_{2} - 3u_{2}''y_{3} - 2u_{2}'y_{4} + u_{2}y_{2}''' + 3u_{2}y_{3}'' + 3u_{2}y_{4}' - 6u_{3}''y_{2} - 3u_{3}'y_{3} + u_{3}y_{2}'' + 2u_{3}y_{3}' - 4u_{4}'y_{2} + u_{4}y_{2}' - 6u_{2}^{(5)} - 5u_{3}^{(4)} - 0u_{4}''' - 5u_{5}'' + y_{2}^{(5)} + 5y_{3}^{(4)} + 10y_{4}''' + 10y_{5}'' + 5y_{6}'&= 0\\
	-6u_{2}''y_{2} - 3u_{2}'y_{3} + 3u_{2}y_{2}'' + 3u_{2}y_{3}' - 4u_{3}'y_{2} + 2u_{3}y_{2}' - 5u_{2}^{(4)} - 0u_{3}''' - 5u_{4}'' - 6u_{5}' + 5y_{2}^{(4)} + 10y_{3}''' + 10y_{4}'' + 5y_{5}'&= 0\\
	-4u_{2}'y_{2} + 3u_{2}y_{2}' - 0u_{2}''' - 5u_{3}'' - 6u_{4}' + 10y_{2}''' + 10y_{3}'' + 5y_{4}'&= 0\\
	-5u_{2}'' - 6u_{3}' + 10y_{2}'' + 5y_{3}'&= 0\\
	-6u_{2}' + 5y_{2}'&= 0
\end{array}\right.


#### **2. Computing the basis of $W(L_5)$**

In [83]:
W = R.weight_func([2,3,4,5,0,0,0,0,0,0], [1])
T = [W.homogeneous_monomials(i) for i in range(7)]

In [84]:
for i in range(7):
    show(LatexExpr(r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}"))

\mathcal{T}_{0} = \{1\}

\mathcal{T}_{1} = \{\}

\mathcal{T}_{2} = \{u_{2}\}

\mathcal{T}_{3} = \{u_{2}',u_{3}\}

\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2,u_{4}\}

\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}'u_{2},u_{4}',u_{2}u_{3}^{(0)},u_{5}\}

\mathcal{T}_{6} = \{u_{2}^{(4)},u_{3}''',u_{2}''u_{2},u_{2}'^2,u_{4}'',u_{2}'u_{3},u_{2}u_{3}',u_{5}',u_{2}^3,u_{2}u_{4}^{(0)},u_{3}^2\}

In [85]:
print(LatexExpr(
    r"\begin{itemize}" + "\n\t" + r"\item $" + ("\n\t" + r"\item $").join(
        r"\mathcal{T}_{" + str(i) + r"} = \{" + ",".join(poly_to_latex(t) for t in T[i]) +  r"\}$"
    for i in range(7)) + "\n" + r"\end{itemize}"))

\begin{itemize}
	\item $\mathcal{T}_{0} = \{1\}$
	\item $\mathcal{T}_{1} = \{\}$
	\item $\mathcal{T}_{2} = \{u_{2}\}$
	\item $\mathcal{T}_{3} = \{u_{2}',u_{3}\}$
	\item $\mathcal{T}_{4} = \{u_{2}'',u_{3}',u_{2}^2,u_{4}\}$
	\item $\mathcal{T}_{5} = \{u_{2}''',u_{3}'',u_{2}'u_{2},u_{4}',u_{2}u_{3}^{(0)},u_{5}\}$
	\item $\mathcal{T}_{6} = \{u_{2}^{(4)},u_{3}''',u_{2}''u_{2},u_{2}'^2,u_{4}'',u_{2}'u_{3},u_{2}u_{3}',u_{5}',u_{2}^3,u_{2}u_{4}^{(0)},u_{3}^2\}$
\end{itemize}


In [86]:
## We create now the constants
RwC = DifferentialRing(PolynomialRing(QQ, sum(([f"c_{i}_{j}" for j in range(len(T[i]))] for i in range(2,7)), [])))
C = [[],[]] + [[RwC(f"c_{i}_{j}") for j in range(len(T[i]))] for i in range(2,7)]

In [87]:
y_subs = {f"y{i}" : sum(c*t for (c,t) in zip(C[i], T[i])) for i in range(2,7)}
print(LatexExpr(
    r"\begin{array}{rcl}" + "\n\t" + "\\\\\n\t".join(poly_to_latex(k) + r" & \rightarrow & " + poly_to_latex(v) for (k,v) in y_subs.items()) + "\n" + r"\end{array}"))

\begin{array}{rcl}
	y_{2} & \rightarrow & c_{2,0}u_{2}\\
	y_{3} & \rightarrow & c_{3,0}u_{2}' + c_{3,1}u_{3}\\
	y_{4} & \rightarrow & c_{4,2}u_{2}^2 + c_{4,0}u_{2}'' + c_{4,1}u_{3}' + c_{4,3}u_{4}\\
	y_{5} & \rightarrow & c_{5,2}u_{2}'u_{2} + c_{5,4}u_{2}u_{3}^{(0)} + c_{5,0}u_{2}''' + c_{5,1}u_{3}'' + c_{5,3}u_{4}' + c_{5,5}u_{5}\\
	y_{6} & \rightarrow & c_{6,8}u_{2}^3 + c_{6,2}u_{2}''u_{2} + c_{6,3}u_{2}'^2 + c_{6,5}u_{2}'u_{3} + c_{6,6}u_{2}u_{3}' + c_{6,9}u_{2}u_{4}^{(0)} + c_{6,10}u_{3}^2 + c_{6,0}u_{2}^{(4)} + c_{6,1}u_{3}''' + c_{6,4}u_{4}'' + c_{6,7}u_{5}'
\end{array}


In [88]:
## We compute the linear system for each $m$
lin_systems = [sum(([RwC.wrapped(c) for c in el(**y_subs).coefficients()] for el in diff_systems[m]), []) for m in range(7)]

In [89]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[2]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	5c_{2,0} & = -2
\end{array}\right.

In [90]:
print(_)

\left\{\begin{array}{rl}
	5c_{2,0} & = -2
\end{array}\right.


In [91]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[3]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	10c_{2,0} + 5c_{3,0} & = -3\\
	5c_{3,1} & = -3\\
	5c_{2,0} & = -3
\end{array}\right.

In [92]:
print(_)

\left\{\begin{array}{rl}
	10c_{2,0} + 5c_{3,0} & = -3\\
	5c_{3,1} & = -3\\
	5c_{2,0} & = -3
\end{array}\right.


In [93]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[4]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	c_{2,0} + 10c_{4,2} & = 0\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = -4\\
	10c_{3,1} + 5c_{4,1} & = -6\\
	5c_{4,3} & = -4\\
	10c_{2,0} + 5c_{3,0} & = -6\\
	5c_{3,1} & = -4\\
	5c_{2,0} & = -4
\end{array}\right.

In [94]:
print(_)

\left\{\begin{array}{rl}
	c_{2,0} + 10c_{4,2} & = 0\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = -4\\
	10c_{3,1} + 5c_{4,1} & = -6\\
	5c_{4,3} & = -4\\
	10c_{2,0} + 5c_{3,0} & = -6\\
	5c_{3,1} & = -4\\
	5c_{2,0} & = -4
\end{array}\right.


In [95]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[5]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-2c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	2c_{2,0} - 2c_{3,1} + 5c_{5,4} & = 0\\
	-3c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = -5\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = -10\\
	10c_{4,3} + 5c_{5,3} & = -10\\
	5c_{5,5} & = -5\\
	10c_{4,2} & = 0\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = -10\\
	10c_{3,1} + 5c_{4,1} & = -10\\
	5c_{4,3} & = -5\\
	10c_{2,0} + 5c_{3,0} & = -10\\
	5c_{3,1} & = -5\\
	5c_{2,0} & = -5
\end{array}\right.

In [96]:
print(_)

\left\{\begin{array}{rl}
	3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-2c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	2c_{2,0} - 2c_{3,1} + 5c_{5,4} & = 0\\
	-3c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = -5\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = -10\\
	10c_{4,3} + 5c_{5,3} & = -10\\
	5c_{5,5} & = -5\\
	10c_{4,2} & = 0\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = -10\\
	10c_{3,1} + 5c_{4,1} & = -10\\
	5c_{4,3} & = -5\\
	10c_{2,0} + 5c_{3,0} & = -10\\
	5c_{3,1} & = -5\\
	5c_{2,0} & = -5
\end{array}\right.


In [97]:
LatexExpr(
    r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(
        poly_to_latex(equ-equ.constant_coefficient()) + r" & = " + poly_to_latex(equ.constant_coefficient()) 
        for equ in lin_systems[6]) + "\n" r"\end{array}\right.")

\left\{\begin{array}{rl}
	4c_{4,2} + 15c_{6,8} & = 0\\
	-3c_{2,0} + 3c_{3,0} + 3c_{4,0} + 20c_{4,2} + 10c_{5,2} + 5c_{6,2} & = 0\\
	-3c_{3,0} - 2c_{4,0} + 60c_{4,2} + 30c_{5,2} + 5c_{6,2} + 10c_{6,3} & = 0\\
	c_{2,0} + 2c_{3,0} - 3c_{3,1} + 10c_{5,4} + 5c_{6,5} & = 0\\
	-3c_{3,0} - 2c_{4,1} + 20c_{5,4} + 5c_{6,5} + 5c_{6,6} & = 0\\
	c_{2,0} - 2c_{4,3} + 5c_{6,9} & = 0\\
	-6c_{2,0} + 3c_{3,1} + 3c_{4,1} + 10c_{5,4} + 5c_{6,6} & = 0\\
	-4c_{2,0} + 3c_{4,3} + 5c_{6,9} & = 0\\
	-c_{3,1} + 10c_{6,10} & = 0\\
	c_{2,0} + 5c_{3,0} + 10c_{4,0} + 10c_{5,0} + 5c_{6,0} & = -6\\
	5c_{3,1} + 10c_{4,1} + 10c_{5,1} + 5c_{6,1} & = -15\\
	10c_{4,3} + 10c_{5,3} + 5c_{6,4} & = -20\\
	10c_{5,5} + 5c_{6,7} & = -15\\
	-3c_{2,0} + 3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	2c_{2,0} - 3c_{3,1} + 5c_{5,4} & = 0\\
	-4c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = -15\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = -20\\
	10c_{4,3} + 5c_{5,3} & = -15\\
	5c_{5,5} & = -6\\
	-c_{2,0} + 10c_{4,2} & = 0\\
	10c_{2,0} + 10c_{3,0} + 5c_{4,0} & = -20\\
	10c_{3,1} + 5c_{4,1} & = -15\\
	5c_{4,3} & = -6\\
	10c_{2,0} + 5c_{3,0} & = -15\\
	5c_{3,1} & = -6\\
	5c_{2,0} & = -6
\end{array}\right.

In [98]:
print(_)

\left\{\begin{array}{rl}
	4c_{4,2} + 15c_{6,8} & = 0\\
	-3c_{2,0} + 3c_{3,0} + 3c_{4,0} + 20c_{4,2} + 10c_{5,2} + 5c_{6,2} & = 0\\
	-3c_{3,0} - 2c_{4,0} + 60c_{4,2} + 30c_{5,2} + 5c_{6,2} + 10c_{6,3} & = 0\\
	c_{2,0} + 2c_{3,0} - 3c_{3,1} + 10c_{5,4} + 5c_{6,5} & = 0\\
	-3c_{3,0} - 2c_{4,1} + 20c_{5,4} + 5c_{6,5} + 5c_{6,6} & = 0\\
	c_{2,0} - 2c_{4,3} + 5c_{6,9} & = 0\\
	-6c_{2,0} + 3c_{3,1} + 3c_{4,1} + 10c_{5,4} + 5c_{6,6} & = 0\\
	-4c_{2,0} + 3c_{4,3} + 5c_{6,9} & = 0\\
	-c_{3,1} + 10c_{6,10} & = 0\\
	c_{2,0} + 5c_{3,0} + 10c_{4,0} + 10c_{5,0} + 5c_{6,0} & = -6\\
	5c_{3,1} + 10c_{4,1} + 10c_{5,1} + 5c_{6,1} & = -15\\
	10c_{4,3} + 10c_{5,3} + 5c_{6,4} & = -20\\
	10c_{5,5} + 5c_{6,7} & = -15\\
	-3c_{2,0} + 3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	-3c_{3,0} + 20c_{4,2} + 5c_{5,2} & = 0\\
	2c_{2,0} - 3c_{3,1} + 5c_{5,4} & = 0\\
	-4c_{2,0} + 3c_{3,1} + 5c_{5,4} & = 0\\
	5c_{2,0} + 10c_{3,0} + 10c_{4,0} + 5c_{5,0} & = -15\\
	10c_{3,1} + 10c_{4,1} + 5c_{5,1} & = -20\\
	10c_{4,3} + 5c_{5,3}

In [99]:
solutions = [ideal(system).groebner_basis() if len(system) > 0 else [] for system in lin_systems]

In [100]:
for i in range(len(solutions)):
    show(i, LatexExpr(r"\qquad\longrightarrow\qquad"), [LatexExpr(poly_to_latex(sol)) for sol in solutions[i]])

0 \qquad\longrightarrow\qquad []

1 \qquad\longrightarrow\qquad []

2 \qquad\longrightarrow\qquad [c_{2,0} - \frac{2}{5}]

3 \qquad\longrightarrow\qquad [c_{2,0} - \frac{3}{5}, c_{3,0} + \frac{3}{5}, c_{3,1} - \frac{3}{5}]

4 \qquad\longrightarrow\qquad [c_{2,0} - \frac{4}{5},
 c_{3,0} + \frac{2}{5},
 c_{3,1} - \frac{4}{5},
 c_{4,0},
 c_{4,1} + \frac{2}{5},
 c_{4,2} + \frac{2}{25},
 c_{4,3} - \frac{4}{5}]

5 \qquad\longrightarrow\qquad [c_{2,0} - 1,
 c_{3,0},
 c_{3,1} - 1,
 c_{4,0},
 c_{4,1},
 c_{4,2},
 c_{4,3} - 1,
 c_{5,0},
 c_{5,1},
 c_{5,2},
 c_{5,3},
 c_{5,4},
 c_{5,5} - 1]

6 \qquad\longrightarrow\qquad [c_{2,0} - \frac{6}{5},
 c_{3,0} - \frac{3}{5},
 c_{3,1} - \frac{6}{5},
 c_{4,0} - \frac{2}{5},
 c_{4,1} - \frac{3}{5},
 c_{4,2} - \frac{3}{25},
 c_{4,3} - \frac{6}{5},
 c_{5,0} + \frac{1}{5},
 c_{5,1} - \frac{2}{5},
 c_{5,2} + \frac{3}{25},
 c_{5,3} - \frac{3}{5},
 c_{5,4} - \frac{6}{25},
 c_{5,5} - \frac{6}{5},
 c_{6,0} + \frac{1}{25},
 c_{6,1} + \frac{1}{5},
 c_{6,2} + \frac{3}{25},
 c_{6,3} + \frac{1}{25},
 c_{6,4} - \frac{2}{5},
 c_{6,5} + \frac{6}{25},
 c_{6,6} + \frac{3}{25},
 c_{6,7} - \frac{3}{5},
 c_{6,8} + \frac{4}{125},
 c_{6,9} - \frac{6}{25},
 c_{6,10} - \frac{3}{25}]

#### **3. Computing the hierarchy equations**

In [101]:
y_subs_true = [dict(),dict()] + [{f"y{i}" : sum(QQ(RwC.wrapped(c).reduce(solutions[m]))*t for (c,t) in zip(C[i], T[i])) for i in range(2,m+1)} for m in range(2,7)]

In [102]:
H = [[lie_brackets[m].coefficient(z[i])(**y_subs_true[m]) for i in range(L.order(z)-1)] for m in range(7)]

##### m = 0

In [103]:
m = 0
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.

In [104]:
print(_)

\left\{\begin{array}{rl}
	H_{0,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{0,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.


##### m = 1

In [105]:
m=1
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3,u_4,u_5) = & -u_{5}'\\
	H_{1,1}(u_2,u_3,u_4,u_5) = & -u_{4}'\\
	H_{1,2}(u_2,u_3,u_4,u_5) = & -u_{3}'\\
	H_{1,3}(u_2,u_3,u_4,u_5) = & -u_{2}'
\end{array}\right.

In [106]:
print(_)

\left\{\begin{array}{rl}
	H_{1,0}(u_2,u_3,u_4,u_5) = & -u_{5}'\\
	H_{1,1}(u_2,u_3,u_4,u_5) = & -u_{4}'\\
	H_{1,2}(u_2,u_3,u_4,u_5) = & -u_{3}'\\
	H_{1,3}(u_2,u_3,u_4,u_5) = & -u_{2}'
\end{array}\right.


##### m = 2

In [107]:
m=2
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3,u_4,u_5) = & \frac{2}{5}u_{2}'''u_{2} + \frac{2}{5}u_{2}''u_{3} + \frac{2}{5}u_{2}'u_{4} + \frac{2}{5}u_{2}^{(5)} - u_{5}''\\
	H_{2,1}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}''u_{2} + \frac{4}{5}u_{2}'u_{3} + 2u_{2}^{(4)} - u_{4}'' - 2u_{5}'\\
	H_{2,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}'u_{2} + 4u_{2}''' - u_{3}'' - 2u_{4}'\\
	H_{2,3}(u_2,u_3,u_4,u_5) = & 3u_{2}'' - 2u_{3}'
\end{array}\right.

In [108]:
print(_)

\left\{\begin{array}{rl}
	H_{2,0}(u_2,u_3,u_4,u_5) = & \frac{2}{5}u_{2}'''u_{2} + \frac{2}{5}u_{2}''u_{3} + \frac{2}{5}u_{2}'u_{4} + \frac{2}{5}u_{2}^{(5)} - u_{5}''\\
	H_{2,1}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}''u_{2} + \frac{4}{5}u_{2}'u_{3} + 2u_{2}^{(4)} - u_{4}'' - 2u_{5}'\\
	H_{2,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}'u_{2} + 4u_{2}''' - u_{3}'' - 2u_{4}'\\
	H_{2,3}(u_2,u_3,u_4,u_5) = & 3u_{2}'' - 2u_{3}'
\end{array}\right.


##### m = 3

In [109]:
m=3
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3,u_4,u_5) = & -5u_{2}^{(4)}u_{2} - 5u_{2}'''u_{3} - 5u_{2}''u_{4} + \frac{3}{5}u_{2}u_{3}''' - 5u_{2}u_{5}' + \frac{3}{5}u_{3}''u_{3} + \frac{3}{5}u_{3}'u_{4} - 5u_{2}^{(6)} + \frac{3}{5}u_{3}^{(5)} - u_{5}'''\\
	H_{3,1}(u_2,u_3,u_4,u_5) = & -5u_{2}'''u_{2} - 5u_{2}''u_{3} + \frac{3}{5}u_{2}'u_{4} + \frac{9}{5}u_{2}u_{3}'' - 5u_{2}u_{4}' + \frac{6}{5}u_{3}'u_{3} - 5u_{2}^{(5)} + 3u_{3}^{(4)} - u_{4}''' - 3u_{5}''\\
	H_{3,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}'u_{3} + \frac{6}{5}u_{2}u_{3}' - 3u_{2}^{(4)} + 5u_{3}''' - 3u_{4}'' - 3u_{5}'\\
	H_{3,3}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}'u_{2} - u_{2}''' + 3u_{3}'' - 3u_{4}'
\end{array}\right.

In [110]:
print(_)

\left\{\begin{array}{rl}
	H_{3,0}(u_2,u_3,u_4,u_5) = & -5u_{2}^{(4)}u_{2} - 5u_{2}'''u_{3} - 5u_{2}''u_{4} + \frac{3}{5}u_{2}u_{3}''' - 5u_{2}u_{5}' + \frac{3}{5}u_{3}''u_{3} + \frac{3}{5}u_{3}'u_{4} - 5u_{2}^{(6)} + \frac{3}{5}u_{3}^{(5)} - u_{5}'''\\
	H_{3,1}(u_2,u_3,u_4,u_5) = & -5u_{2}'''u_{2} - 5u_{2}''u_{3} + \frac{3}{5}u_{2}'u_{4} + \frac{9}{5}u_{2}u_{3}'' - 5u_{2}u_{4}' + \frac{6}{5}u_{3}'u_{3} - 5u_{2}^{(5)} + 3u_{3}^{(4)} - u_{4}''' - 3u_{5}''\\
	H_{3,2}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}'u_{3} + \frac{6}{5}u_{2}u_{3}' - 3u_{2}^{(4)} + 5u_{3}''' - 3u_{4}'' - 3u_{5}'\\
	H_{3,3}(u_2,u_3,u_4,u_5) = & \frac{6}{5}u_{2}'u_{2} - u_{2}''' + 3u_{3}'' - 3u_{4}'
\end{array}\right.


##### m = 4

In [111]:
m=4
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3,u_4,u_5) = & -5u_{2}'''u_{2}^2 - 5u_{2}''u_{2}'u_{2} - 5u_{2}''u_{2}u_{3}^{(0)} - 5u_{2}'^2u_{3} - 5u_{2}'u_{2}u_{4}^{(0)} - 5u_{2}^{(5)}u_{2} - 5u_{2}^{(4)}u_{2}' - 5u_{2}'''u_{2}'' + \frac{2}{5}u_{2}'u_{5}^{(1)} - 5u_{2}u_{3}^{(4)} + \frac{4}{5}u_{2}u_{4}''' - 5u_{2}u_{5}'' - 5u_{3}'''u_{3} - 5u_{3}''u_{4} + \frac{4}{5}u_{3}u_{4}'' - 5u_{3}u_{5}' + \frac{4}{5}u_{4}'u_{4} - 5u_{3}^{(6)} + \frac{4}{5}u_{4}^{(5)} - u_{5}^{(4)}\\
	H_{4,1}(u_2,u_3,u_4,u_5) = & -5u_{2}''u_{2}^2 - 5u_{2}'^2u_{2} - 5u_{2}'u_{2}u_{3}^{(0)} - 5u_{2}^{(4)}u_{2} - 5u_{2}'''u_{2}' - 5u_{2}'''u_{3} - 5u_{2}''^2 - 5u_{2}''u_{4} + \frac{2}{5}u_{2}'u_{4}^{(1)} - 5u_{2}u_{3}''' + \frac{8}{5}u_{2}u_{4}'' - 5u_{2}u_{5}' + \frac{4}{5}u_{3}'u_{4} + \frac{4}{5}u_{3}u_{4}' - 5u_{2}^{(6)} - 5u_{3}^{(5)} + 3u_{4}^{(4)} - 4u_{5}'''\\
	H_{4,2}(u_2,u_3,u_4,u_5) = & -5u_{2}'u_{2}^2 - 2u_{2}'''u_{2} - 5u_{2}''u_{2}' + \frac{2}{5}u_{2}'u_{3}^{(1)} + \frac{4}{5}u_{2}'u_{4} + \frac{2}{5}u_{2}u_{3}'' + \frac{4}{5}u_{2}u_{4}' + \frac{4}{5}u_{3}'u_{3} - 5u_{2}^{(5)} - u_{3}^{(4)} + 4u_{4}''' - 6u_{5}''\\
	H_{4,3}(u_2,u_3,u_4,u_5) = & -5u_{2}''u_{2} - 5u_{2}'^2 + \frac{4}{5}u_{2}'u_{3} + \frac{4}{5}u_{2}u_{3}' - u_{2}^{(4)} + 2u_{4}'' - 4u_{5}'
\end{array}\right.

In [112]:
print(_)

\left\{\begin{array}{rl}
	H_{4,0}(u_2,u_3,u_4,u_5) = & -5u_{2}'''u_{2}^2 - 5u_{2}''u_{2}'u_{2} - 5u_{2}''u_{2}u_{3}^{(0)} - 5u_{2}'^2u_{3} - 5u_{2}'u_{2}u_{4}^{(0)} - 5u_{2}^{(5)}u_{2} - 5u_{2}^{(4)}u_{2}' - 5u_{2}'''u_{2}'' + \frac{2}{5}u_{2}'u_{5}^{(1)} - 5u_{2}u_{3}^{(4)} + \frac{4}{5}u_{2}u_{4}''' - 5u_{2}u_{5}'' - 5u_{3}'''u_{3} - 5u_{3}''u_{4} + \frac{4}{5}u_{3}u_{4}'' - 5u_{3}u_{5}' + \frac{4}{5}u_{4}'u_{4} - 5u_{3}^{(6)} + \frac{4}{5}u_{4}^{(5)} - u_{5}^{(4)}\\
	H_{4,1}(u_2,u_3,u_4,u_5) = & -5u_{2}''u_{2}^2 - 5u_{2}'^2u_{2} - 5u_{2}'u_{2}u_{3}^{(0)} - 5u_{2}^{(4)}u_{2} - 5u_{2}'''u_{2}' - 5u_{2}'''u_{3} - 5u_{2}''^2 - 5u_{2}''u_{4} + \frac{2}{5}u_{2}'u_{4}^{(1)} - 5u_{2}u_{3}''' + \frac{8}{5}u_{2}u_{4}'' - 5u_{2}u_{5}' + \frac{4}{5}u_{3}'u_{4} + \frac{4}{5}u_{3}u_{4}' - 5u_{2}^{(6)} - 5u_{3}^{(5)} + 3u_{4}^{(4)} - 4u_{5}'''\\
	H_{4,2}(u_2,u_3,u_4,u_5) = & -5u_{2}'u_{2}^2 - 2u_{2}'''u_{2} - 5u_{2}''u_{2}' + \frac{2}{5}u_{2}'u_{3}^{(1)} + \frac{4}{5}u_{2}'u_{4} + \frac{2}{5}u_{2}

##### m = 5

In [113]:
m=5
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.

In [114]:
print(_)

\left\{\begin{array}{rl}
	H_{5,0}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,1}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,2}(u_2,u_3,u_4,u_5) = & 0\\
	H_{5,3}(u_2,u_3,u_4,u_5) = & 0
\end{array}\right.


##### m = 6

In [115]:
m=6
LatexExpr(r"\left\{\begin{array}{rl}" + "\n\t" + "\\\\\n\t".join(f"H_\u007b{m},{i}\u007d(u_2,u_3,u_4,u_5) = & " + poly_to_latex(H[m][i]) for i in range(L.order(z)-1)) + "\n" + r"\end{array}\right.")

\left\{\begin{array}{rl}
	H_{6,0}(u_2,u_3,u_4,u_5) = & -5u_{2}'''u_{2}^3 - 5u_{2}''u_{2}'u_{2}^2 - 5u_{2}''u_{2}^2u_{3} - 5u_{2}'^3u_{2} - 5u_{2}'^2u_{2}u_{3}^{(0)} - 5u_{2}'u_{2}^2u_{4} - 5u_{2}^{(5)}u_{2}^2 - 5u_{2}^{(4)}u_{2}'u_{2} - 5u_{2}^{(4)}u_{2}u_{3}^{(0)} - 5u_{2}'''u_{2}''u_{2} - 5u_{2}'''u_{2}'^2 - 5u_{2}'''u_{2}'u_{3} - 5u_{2}'''u_{2}u_{3}' + \frac{3}{25}u_{2}'''u_{2}u_{4}^{(0)} - 5u_{2}'''u_{3}^2 - 5u_{2}''^2u_{2}' - 5u_{2}''^2u_{3} - 5u_{2}''u_{2}'u_{4} - 5u_{2}''u_{2}u_{3}'' + \frac{18}{25}u_{2}''u_{2}u_{4}' - 5u_{2}''u_{3}'u_{3} - 5u_{2}'u_{2}u_{3}''' + \frac{18}{25}u_{2}'u_{2}u_{4}'' + \frac{3}{25}u_{2}'u_{2}u_{5}' - 5u_{2}'u_{3}''u_{3} - 5u_{2}'u_{3}^{(1)}u_{4} + \frac{12}{25}u_{2}'u_{3}u_{4}' + \frac{6}{25}u_{2}'u_{4}^2 - 5u_{2}^2u_{3}^{(4)} + \frac{6}{25}u_{2}^2u_{4}''' - 5u_{2}^2u_{5}'' + \frac{3}{25}u_{2}u_{3}'''u_{3} + \frac{18}{25}u_{2}u_{3}''u_{3}' - 5u_{2}u_{3}''u_{4} + \frac{6}{25}u_{2}u_{3}^{(0)}u_{4}'' - 5u_{2}u_{3}^{(0)}u_{5}' + \frac{6}{25}u_{2}u_{4}'u_{4} + \frac{6}{25}u_{3}''u_{3}^2 + \frac{6}{25}u_{3}'^2u_{3} + \frac{6}{25}u_{3}'u_{3}u_{4}^{(0)} - 5u_{2}^{(7)}u_{2} - 5u_{2}^{(6)}u_{2}' - 5u_{2}^{(6)}u_{3} - 5u_{2}^{(5)}u_{2}'' - 5u_{2}^{(5)}u_{3}' + \frac{1}{5}u_{2}^{(5)}u_{4} - 5u_{2}^{(4)}u_{2}''' - 3u_{2}^{(4)}u_{3}'' + \frac{6}{5}u_{2}^{(4)}u_{4}' - 5u_{2}'''u_{3}^{(3)} + \frac{12}{5}u_{2}'''u_{4}'' + \frac{1}{5}u_{2}'''u_{5}' - 5u_{2}''u_{3}^{(4)} + \frac{12}{5}u_{2}''u_{4}''' - 5u_{2}''u_{5}^{(2)} - 5u_{2}'u_{3}^{(5)} + \frac{6}{5}u_{2}'u_{4}^{(4)} - 5u_{2}'u_{5}''' - 5u_{2}u_{3}^{(6)} + \frac{16}{25}u_{2}u_{4}^{(5)} - 5u_{2}u_{5}^{(4)} + \frac{1}{25}u_{3}^{(5)}u_{3} + \frac{6}{5}u_{3}^{(4)}u_{3}' - 5u_{3}^{(4)}u_{4} + \frac{12}{5}u_{3}'''u_{3}'' - 5u_{3}''u_{5}' - 5u_{3}'u_{5}'' + \frac{2}{5}u_{3}u_{4}^{(4)} - 5u_{3}u_{5}''' + \frac{2}{5}u_{4}'''u_{4} - 5u_{4}'u_{5}^{(1)} - 5u_{4}u_{5}'' - 5u_{5}'u_{5} - 5u_{2}^{(9)} - 5u_{3}^{(8)} + \frac{2}{5}u_{4}^{(7)} - 5u_{5}^{(6)}\\
	H_{6,1}(u_2,u_3,u_4,u_5) = & -5u_{2}''u_{2}^3 - 5u_{2}'^2u_{2}^2 - 5u_{2}'u_{2}^2u_{3} - 5u_{2}^{(4)}u_{2}^2 - 5u_{2}'''u_{2}'u_{2} - 5u_{2}'''u_{2}u_{3}^{(0)} - 5u_{2}''^2u_{2} - 5u_{2}''u_{2}'^2 - 5u_{2}''u_{2}'u_{3} - 5u_{2}''u_{2}u_{3}' + \frac{3}{5}u_{2}''u_{2}u_{4}^{(0)} - 5u_{2}''u_{3}^2 - 5u_{2}'^2u_{4} - 5u_{2}'u_{2}u_{3}'' + \frac{39}{25}u_{2}'u_{2}u_{4}' - 5u_{2}'u_{3}^{(1)}u_{3} + \frac{18}{25}u_{2}'u_{3}u_{4}^{(0)} - 5u_{2}^2u_{3}''' + \frac{3}{5}u_{2}^2u_{4}'' - 5u_{2}^2u_{5}' + \frac{18}{25}u_{2}u_{3}''u_{3} + \frac{18}{25}u_{2}u_{3}'^2 + \frac{6}{25}u_{2}u_{3}'u_{4} + \frac{6}{25}u_{2}u_{3}^{(0)}u_{4}' + \frac{12}{25}u_{3}'u_{3}^2 - 5u_{2}^{(6)}u_{2} - 5u_{2}^{(5)}u_{2}' - 5u_{2}^{(5)}u_{3} - 5u_{2}^{(4)}u_{2}'' - 5u_{2}^{(4)}u_{3}' + u_{2}^{(4)}u_{4} - 5u_{2}'''^2 - 5u_{2}'''u_{3}'' + 5u_{2}'''u_{4}' - 6u_{2}''u_{3}''' + \frac{34}{5}u_{2}''u_{4}^{(2)} - 5u_{2}''u_{5}' - 5u_{2}'u_{3}^{(4)} + \frac{21}{5}u_{2}'u_{4}''' - 5u_{2}'u_{5}'' - 5u_{2}u_{3}^{(5)} + \frac{9}{5}u_{2}u_{4}^{(4)} - 5u_{2}u_{5}''' + \frac{6}{5}u_{3}^{(4)}u_{3} + \frac{24}{5}u_{3}'''u_{3}' + \frac{2}{5}u_{3}'''u_{4} + \frac{18}{5}u_{3}''^2 - 5u_{3}''u_{4}' - 5u_{3}'u_{4}'' - 5u_{3}'u_{5}^{(1)} + \frac{1}{5}u_{3}u_{4}''' - 5u_{3}u_{5}'' - 5u_{4}''u_{4} - 5u_{4}'^2 - 5u_{4}'u_{5} - 5u_{4}u_{5}' - 5u_{2}^{(8)} - 5u_{3}^{(7)} + \frac{8}{5}u_{4}^{(6)} - 5u_{5}^{(5)}\\
	H_{6,2}(u_2,u_3,u_4,u_5) = & -5u_{2}'u_{2}^3 - 5u_{2}'''u_{2}^2 - 5u_{2}''u_{2}'u_{2} - 5u_{2}'^3 + \frac{12}{25}u_{2}'u_{2}u_{3}' + \frac{24}{25}u_{2}'u_{2}u_{4}^{(0)} + \frac{12}{25}u_{2}'u_{3}^2 + \frac{6}{25}u_{2}^2u_{3}'' + \frac{12}{25}u_{2}^2u_{4}' + \frac{24}{25}u_{2}u_{3}'u_{3} - 5u_{2}^{(5)}u_{2} - 5u_{2}^{(4)}u_{2}' - 5u_{2}^{(4)}u_{3} - 5u_{2}'''u_{2}'' - 5u_{2}'''u_{3}' + \frac{14}{5}u_{2}'''u_{4} - 4u_{2}''u_{3}^{(2)} + \frac{32}{5}u_{2}''u_{4}' - 5u_{2}'u_{3}''' + \frac{27}{5}u_{2}'u_{4}'' - 5u_{2}'u_{5}^{(1)} + \frac{9}{5}u_{2}u_{4}''' - 5u_{2}u_{5}'' + \frac{13}{5}u_{3}'''u_{3} + \frac{31}{5}u_{3}''u_{3}' - 5u_{3}''u_{4} - 5u_{3}'u_{4}^{(1)} - 5u_{3}'u_{5

In [116]:
print(_)

\left\{\begin{array}{rl}
	H_{6,0}(u_2,u_3,u_4,u_5) = & -5u_{2}'''u_{2}^3 - 5u_{2}''u_{2}'u_{2}^2 - 5u_{2}''u_{2}^2u_{3} - 5u_{2}'^3u_{2} - 5u_{2}'^2u_{2}u_{3}^{(0)} - 5u_{2}'u_{2}^2u_{4} - 5u_{2}^{(5)}u_{2}^2 - 5u_{2}^{(4)}u_{2}'u_{2} - 5u_{2}^{(4)}u_{2}u_{3}^{(0)} - 5u_{2}'''u_{2}''u_{2} - 5u_{2}'''u_{2}'^2 - 5u_{2}'''u_{2}'u_{3} - 5u_{2}'''u_{2}u_{3}' + \frac{3}{25}u_{2}'''u_{2}u_{4}^{(0)} - 5u_{2}'''u_{3}^2 - 5u_{2}''^2u_{2}' - 5u_{2}''^2u_{3} - 5u_{2}''u_{2}'u_{4} - 5u_{2}''u_{2}u_{3}'' + \frac{18}{25}u_{2}''u_{2}u_{4}' - 5u_{2}''u_{3}'u_{3} - 5u_{2}'u_{2}u_{3}''' + \frac{18}{25}u_{2}'u_{2}u_{4}'' + \frac{3}{25}u_{2}'u_{2}u_{5}' - 5u_{2}'u_{3}''u_{3} - 5u_{2}'u_{3}^{(1)}u_{4} + \frac{12}{25}u_{2}'u_{3}u_{4}' + \frac{6}{25}u_{2}'u_{4}^2 - 5u_{2}^2u_{3}^{(4)} + \frac{6}{25}u_{2}^2u_{4}''' - 5u_{2}^2u_{5}'' + \frac{3}{25}u_{2}u_{3}'''u_{3} + \frac{18}{25}u_{2}u_{3}''u_{3}' - 5u_{2}u_{3}''u_{4} + \frac{6}{25}u_{2}u_{3}^{(0)}u_{4}'' - 5u_{2}u_{3}^{(0)}u_{5}' + \frac{6}{25}u_{2}u_{4}'u_{